<a href="https://colab.research.google.com/github/Manvi-16/Myntra-One-Stop-Fashion-For-All/blob/main/imageprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade -q accelerate bitsandbytes
from transformers import AutoProcessor, LlavaForConditionalGeneration
from transformers import BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)


model_id = "llava-hf/llava-1.5-7b-hf"

processor = AutoProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto")

import requests
from PIL import Image

image1 = Image.open("/content/image_3.jpg")
# image2 = Image.open(requests.get("http://images.cocodataset.org/val2017/000000039769.jpg", stream=True).raw)
display(image1)
# display(image2)


prompts = [
            "USER: <image>\nDescribe in detail with keywords just the clothing article present in this image. Describe both the pant and the shirt \nASSISTANT:",
            # "USER: <image>\nPlease describe this image\nASSISTANT:",
]

inputs = processor(prompts, images=[image1], padding=True, return_tensors="pt").to("cuda")
for k,v in inputs.items():
  print(k,v.shape)


output = model.generate(**inputs, max_new_tokens=50)
generated_text = processor.batch_decode(output, skip_special_tokens=True)
for text in generated_text:
  print(text.split("ASSISTANT:")[-1])